# 커뮤니티 파이프라인 및 구성 요소 로드

## 커뮤니티 파이프라인

> [!TIP] GitHub 이슈 [#841](https://github.com/huggingface/diffusers/issues/841)을 참조하여 모든 사람이 작업을 쉽게 공유할 수 있도록 커뮤니티 파이프라인을 추가하는 이유에 대한 자세한 내용을 확인하세요.

커뮤니티 파이프라인은 원본 논문 구현과 다른 모든 [DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline) 클래스입니다(예: [StableDiffusionControlNetPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/controlnet#diffusers.StableDiffusionControlNetPipeline)은 [ControlNet 조건화를 사용한 텍스트-이미지 생성](https://huggingface.co/papers/2302.05543) 논문에 해당합니다). 추가 기능을 제공하거나 파이프라인의 원본 구현을 확장합니다.

[Marigold Depth Estimation](https://github.com/huggingface/diffusers/tree/main/examples/community#marigold-depth-estimation) 또는 [InstantID](https://github.com/huggingface/diffusers/tree/main/examples/community#instantid-pipeline)와 같은 멋진 커뮤니티 파이프라인이 많이 있으며, 모든 공식 커뮤니티 파이프라인은 [여기](https://github.com/huggingface/diffusers/tree/main/examples/community)에서 찾을 수 있습니다.

커뮤니티 파이프라인에는 Hugging Face Hub에 저장된 파이프라인과 Diffusers GitHub 리포지토리에 저장된 파이프라인의 두 가지 유형이 있습니다. Hub 파이프라인은 완전히 사용자 지정할 수 있는 반면(스케줄러, 모델, 파이프라인 코드 등) Diffusers GitHub 파이프라인은 사용자 지정 파이프라인 코드로만 제한됩니다.

|                | GitHub 커뮤니티 파이프라인                                                                                        | HF Hub 커뮤니티 파이프라인                                                                 |
|----------------|------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|
| 사용법          | 동일                                                                                                             | 동일                                                                                      |
| 검토 프로세스 | GitHub에서 풀 리퀘스트를 열고 병합하기 전에 Diffusers 팀의 검토 프로세스를 거칩니다. 더 느릴 수 있습니다. | 검토 없이 Hub 리포지토리에 직접 업로드합니다. 이것이 가장 빠른 워크플로입니다.      |
| 가시성     | 공식 Diffusers 리포지토리 및 설명서에 포함됨                                                  | HF Hub 프로필에 포함되며 가시성을 얻기 위해 자체 사용/홍보에 의존합니다. |

<hfoptions id="community">
<hfoption id="Hub pipelines">

Hugging Face Hub 커뮤니티 파이프라인을 로드하려면 커뮤니티 파이프라인의 리포지토리 ID를 `custom_pipeline` 인수에 전달하고 파이프라인 가중치 및 구성 요소를 로드할 모델 리포지토리를 전달합니다. 예를 들어 아래 예제에서는 [hf-internal-testing/diffusers-dummy-pipeline](https://huggingface.co/hf-internal-testing/diffusers-dummy-pipeline/blob/main/pipeline.py)에서 더미 파이프라인을 로드하고 [google/ddpm-cifar10-32](https://huggingface.co/google/ddpm-cifar10-32)에서 파이프라인 가중치 및 구성 요소를 로드합니다.

> [!WARNING]
> Hugging Face Hub에서 커뮤니티 파이프라인을 로드하면 로드하는 코드가 안전하다고 신뢰하는 것입니다. 로드하고 자동으로 실행하기 전에 온라인에서 코드를 검사해야 합니다!

In [ ]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained(
    "google/ddpm-cifar10-32", custom_pipeline="hf-internal-testing/diffusers-dummy-pipeline", use_safetensors=True
)

</hfoption>
<hfoption id="GitHub pipelines">

GitHub 커뮤니티 파이프라인을 로드하려면 커뮤니티 파이프라인의 리포지토리 ID를 `custom_pipeline` 인수에 전달하고 파이프라인 가중치 및 구성 요소를 로드할 모델 리포지토리를 전달합니다. 모델 구성 요소를 직접 로드할 수도 있습니다. 아래 예제에서는 커뮤니티 [CLIP Guided Stable Diffusion](https://github.com/huggingface/diffusers/tree/main/examples/community#clip-guided-stable-diffusion) 파이프라인과 CLIP 모델 구성 요소를 로드합니다.

In [ ]:
from diffusers import DiffusionPipeline
from transformers import CLIPImageProcessor, CLIPModel

clip_model_id = "laion/CLIP-ViT-B-32-laion2B-s34B-b79K"

feature_extractor = CLIPImageProcessor.from_pretrained(clip_model_id)
clip_model = CLIPModel.from_pretrained(clip_model_id)

pipeline = DiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    custom_pipeline="clip_guided_stable_diffusion",
    clip_model=clip_model,
    feature_extractor=feature_extractor,
    use_safetensors=True,
)

</hfoption>
</hfoptions>

### 로컬 파일에서 로드

커뮤니티 파이프라인은 대신 파일 경로를 전달하면 로컬 파일에서도 로드할 수 있습니다. 전달된 디렉터리의 경로에는 파이프라인 클래스가 포함된 pipeline.py 파일이 포함되어야 합니다.

In [ ]:
pipeline = DiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    custom_pipeline="./path/to/pipeline_directory/",
    clip_model=clip_model,
    feature_extractor=feature_extractor,
    use_safetensors=True,
)

### 특정 버전에서 로드

기본적으로 커뮤니티 파이프라인은 최신 안정 버전의 Diffusers에서 로드됩니다. 다른 버전에서 커뮤니티 파이프라인을 로드하려면 `custom_revision` 매개변수를 사용합니다.

<hfoptions id="version">
<hfoption id="main">

예를 들어, 메인 브랜치에서 로드하려면:

In [ ]:
pipeline = DiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    custom_pipeline="clip_guided_stable_diffusion",
    custom_revision="main",
    clip_model=clip_model,
    feature_extractor=feature_extractor,
    use_safetensors=True,
)

</hfoption>
<hfoption id="older version">

예를 들어, v0.25.0과 같은 이전 버전의 Diffusers에서 로드하려면:

In [ ]:
pipeline = DiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    custom_pipeline="clip_guided_stable_diffusion",
    custom_revision="v0.25.0",
    clip_model=clip_model,
    feature_extractor=feature_extractor,
    use_safetensors=True,
)

</hfoption>
</hfoptions>

### from_pipe로 로드

커뮤니티 파이프라인은 [from_pipe()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pipe) 메서드를 사용하여 로드할 수도 있으며, 이를 통해 추가적인 메모리 오버헤드 없이 여러 파이프라인을 로드하고 재사용할 수 있습니다([파이프라인 재사용](https://huggingface.co/docs/diffusers/main/en/using-diffusers/./loading#reuse-a-pipeline) 가이드에서 자세히 알아보세요). 메모리 요구 사항은 로드된 가장 큰 단일 파이프라인에 의해 결정됩니다.

예를 들어, Stable Diffusion 파이프라인에서 [가중치를 사용한 긴 프롬프트](https://github.com/huggingface/diffusers/tree/main/examples/community#long-prompt-weighting-stable-diffusion)를 지원하는 커뮤니티 파이프라인을 로드해 보겠습니다.

In [ ]:
import torch
from diffusers import DiffusionPipeline

pipe_sd = DiffusionPipeline.from_pretrained("emilianJR/CyberRealistic_V3", torch_dtype=torch.float16)
pipe_sd.to("cuda")
# 긴 프롬프트 가중치 파이프라인 로드
pipe_lpw = DiffusionPipeline.from_pipe(
    pipe_sd,
    custom_pipeline="lpw_stable_diffusion",
).to("cuda")

prompt = "cat, hiding in the leaves, ((rain)), zazie rainyday, beautiful eyes, macro shot, colorful details, natural lighting, amazing composition, subsurface scattering, amazing textures, filmic, soft light, ultra-detailed eyes, intricate details, detailed texture, light source contrast, dramatic shadows, cinematic light, depth of field, film grain, noise, dark background, hyperrealistic dslr film still, dim volumetric cinematic lighting"
neg_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime, mutated hands and fingers:1.4), (deformed, distorted, disfigured:1.3), poorly drawn, bad anatomy, wrong anatomy, extra limb, missing limb, floating limbs, disconnected limbs, mutation, mutated, ugly, disgusting, amputation"
generator = torch.Generator(device="cpu").manual_seed(20)
out_lpw = pipe_lpw(
    prompt,
    negative_prompt=neg_prompt,
    width=512,
    height=512,
    max_embeddings_multiples=3,
    num_inference_steps=50,
    generator=generator,
    ).images[0]
out_lpw

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/from_pipe_lpw.png" />
    <figcaption class="mt-2 text-center text-sm text-gray-500">긴 프롬프트 가중치를 사용한 스테이블 디퓨전</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/from_pipe_non_lpw.png" />
    <figcaption class="mt-2 text-center text-sm text-gray-500">스테이블 디퓨전</figcaption>
  </div>
</div>

## 예제 커뮤니티 파이프라인

커뮤니티 파이프라인은 새롭고 독특한 기능으로 원래 파이프라인의 기능을 확장하는 정말 재미있고 창의적인 방법입니다. [diffusers/examples/community](https://github.com/huggingface/diffusers/tree/main/examples/community) 폴더에서 모든 커뮤니티 파이프라인과 사용 방법에 대한 추론 및 학습 예제를 찾을 수 있습니다.

이 섹션에서는 몇 가지 커뮤니티 파이프라인을 소개하며, 이를 통해 자신만의 파이프라인을 만드는 데 영감을 얻을 수 있기를 바랍니다(커뮤니티 파이프라인에 대한 PR을 자유롭게 열고 검토를 위해 저희에게 핑을 보내주세요)!

> [!TIP]
> [from_pipe()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pipe) 메서드는 사전 학습된 가중치가 없고 Stable Diffusion 또는 Stable Diffusion XL과 같은 기존 파이프라인 위에 기능을 추가하는 경우가 많기 때문에 커뮤니티 파이프라인을 로드하는 데 특히 유용합니다. [from_pipe()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pipe) 메서드에 대한 자세한 내용은 [from_pipe로 로드](https://huggingface.co/docs/diffusers/main/en/using-diffusers/custom_pipeline_overview#load-with-from_pipe) 섹션에서 확인할 수 있습니다.

<hfoptions id="community">
<hfoption id="Marigold">

[Marigold](https://marigoldmonodepth.github.io/)는 확산 모델에 있는 풍부하고 고유한 시각적 지식을 사용하는 깊이 추정 확산 파이프라인입니다. 입력 이미지를 가져와 노이즈를 제거하고 깊이 맵으로 디코딩합니다. Marigold는 이전에 본 적 없는 이미지에서도 잘 작동합니다.

In [ ]:
import torch
from PIL import Image
from diffusers import DiffusionPipeline
from diffusers.utils import load_image

pipeline = DiffusionPipeline.from_pretrained(
    "prs-eth/marigold-lcm-v1-0",
    custom_pipeline="marigold_depth_estimation",
    torch_dtype=torch.float16,
    variant="fp16",
)

pipeline.to("cuda")
image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/community-marigold.png")
output = pipeline(
    image,
    denoising_steps=4,
    ensemble_size=5,
    processing_res=768,
    match_input_res=True,
    batch_size=0,
    seed=33,
    color_map="Spectral",
    show_progress_bar=True,
)
depth_colored: Image.Image = output.depth_colored
depth_colored.save("./depth_colored.png")

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/community-marigold.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">원본 이미지</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/marigold-depth.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">색상화된 깊이 이미지</figcaption>
  </div>
</div>

</hfoption>
<hfoption id="HD-Painter">

[HD-Painter](https://hf.co/papers/2312.14091)는 고해상도 인페인팅 파이프라인입니다. 프롬프트와 인페인트할 영역을 더 잘 정렬하기 위해 *Prompt-Aware Introverted Attention (PAIntA)* 레이어를 도입하고, 잠재 변수를 프롬프트에 더 잘 정렬하고 학습된 도메인 내에 유지하여 사실적인 이미지를 생성하기 위해 *Reweighting Attention Score Guidance (RASG)*를 도입합니다.

In [ ]:
import torch
from diffusers import DiffusionPipeline, DDIMScheduler
from diffusers.utils import load_image

pipeline = DiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5-inpainting",
    custom_pipeline="hd_painter"
)
pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/hd-painter.jpg")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/hd-painter-mask.png")
prompt = "football"
image = pipeline(prompt, init_image, mask_image, use_rasg=True, use_painta=True, generator=torch.manual_seed(0)).images[0]
image

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/hd-painter.jpg"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">원본 이미지</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/hd-painter-output.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">생성된 이미지</figcaption>
  </div>
</div>

</hfoption>
</hfoptions>

## 커뮤니티 구성 요소

커뮤니티 구성 요소를 사용하면 사용자는 Diffusers의 일부가 아닌 사용자 지정 구성 요소가 있을 수 있는 파이프라인을 구축할 수 있습니다. 파이프라인에 Diffusers가 아직 지원하지 않는 사용자 지정 구성 요소가 있는 경우 해당 구현을 Python 모듈로 제공해야 합니다. 이러한 사용자 지정 구성 요소는 VAE, UNet 및 스케줄러일 수 있습니다. 대부분의 경우 텍스트 인코더는 Transformers 라이브러리에서 가져옵니다. 파이프라인 코드 자체도 사용자 지정할 수 있습니다.

이 섹션에서는 사용자가 커뮤니티 구성 요소를 사용하여 커뮤니티 파이프라인을 구축하는 방법을 보여줍니다.

[showlab/show-1-base](https://huggingface.co/showlab/show-1-base) 파이프라인 체크포인트를 예로 사용합니다.

1. Transformers에서 텍스트 인코더를 가져오고 로드합니다.

In [ ]:
from transformers import T5Tokenizer, T5EncoderModel

pipe_id = "showlab/show-1-base"
tokenizer = T5Tokenizer.from_pretrained(pipe_id, subfolder="tokenizer")
text_encoder = T5EncoderModel.from_pretrained(pipe_id, subfolder="text_encoder")

2. 스케줄러 로드:

In [ ]:
from diffusers import DPMSolverMultistepScheduler

scheduler = DPMSolverMultistepScheduler.from_pretrained(pipe_id, subfolder="scheduler")

3. 이미지 프로세서 로드:

In [ ]:
from transformers import CLIPImageProcessor

feature_extractor = CLIPImageProcessor.from_pretrained(pipe_id, subfolder="feature_extractor")

<Tip warning={true}>

4단계와 5단계에서 사용자 지정 [UNet](https://github.com/showlab/Show-1/blob/main/showone/models/unet_3d_condition.py) 및 [파이프라인](https://huggingface.co/sayakpaul/show-1-base-with-code/blob/main/unet/showone_unet_3d_condition.py) 구현은 이 예제가 작동하도록 해당 파일에 표시된 형식과 일치해야 합니다.

</Tip>

4. 이제 [사용자 지정 UNet](https://github.com/showlab/Show-1/blob/main/showone/models/unet_3d_condition.py)을 로드합니다. 이 예제에서는 편의를 위해 이미 [showone_unet_3d_condition.py](https://huggingface.co/sayakpaul/show-1-base-with-code/blob/main/unet/showone_unet_3d_condition.py)에 구현되어 있습니다. [UNet3DConditionModel](https://huggingface.co/docs/diffusers/main/en/api/models/unet3d-cond#diffusers.UNet3DConditionModel) 클래스 이름이 `ShowOneUNet3DConditionModel`로 변경된 것을 알 수 있습니다. 이는 [UNet3DConditionModel](https://huggingface.co/docs/diffusers/main/en/api/models/unet3d-cond#diffusers.UNet3DConditionModel)이 이미 Diffusers에 존재하기 때문입니다. `ShowOneUNet3DConditionModel` 클래스에 필요한 모든 구성 요소는 showone_unet_3d_condition.py에 배치해야 합니다.

    이 작업이 완료되면 UNet을 초기화할 수 있습니다.

    ```python
    from showone_unet_3d_condition import ShowOneUNet3DConditionModel

    unet = ShowOneUNet3DConditionModel.from_pretrained(pipe_id, subfolder="unet")
    ```

5. 마지막으로 사용자 지정 파이프라인 코드를 로드합니다. 이 예제에서는 이미 [pipeline_t2v_base_pixel.py](https://huggingface.co/sayakpaul/show-1-base-with-code/blob/main/pipeline_t2v_base_pixel.py)에 만들어져 있습니다. 이 스크립트에는 텍스트에서 비디오를 생성하기 위한 사용자 지정 `TextToVideoIFPipeline` 클래스가 포함되어 있습니다. 사용자 지정 UNet과 마찬가지로 사용자 지정 파이프라인이 작동하는 데 필요한 모든 코드는 pipeline_t2v_base_pixel.py에 있어야 합니다.

모든 것이 준비되면 `ShowOneUNet3DConditionModel`을 사용하여 `TextToVideoIFPipeline`을 초기화할 수 있습니다.

In [ ]:
from pipeline_t2v_base_pixel import TextToVideoIFPipeline
import torch

pipeline = TextToVideoIFPipeline(
    unet=unet,
    text_encoder=text_encoder,
    tokenizer=tokenizer,
    scheduler=scheduler,
    feature_extractor=feature_extractor
)
pipeline = pipeline.to(device="cuda")
pipeline.torch_dtype = torch.float16

커뮤니티와 공유하기 위해 파이프라인을 Hub에 푸시하세요!

In [ ]:
pipeline.push_to_hub("custom-t2v-pipeline")

파이프라인이 성공적으로 푸시된 후 몇 가지 변경 사항을 적용해야 합니다.

1. [model_index.json](https://huggingface.co/sayakpaul/show-1-base-with-code/blob/main/model_index.json#L2)의 `_class_name` 속성을 `"pipeline_t2v_base_pixel"` 및 `"TextToVideoIFPipeline"`으로 변경합니다.
2. `showone_unet_3d_condition.py`를 [unet](https://huggingface.co/sayakpaul/show-1-base-with-code/blob/main/unet/showone_unet_3d_condition.py) 하위 폴더에 업로드합니다.
3. `pipeline_t2v_base_pixel.py`를 파이프라인 [리포지토리](https://huggingface.co/sayakpaul/show-1-base-with-code/tree/main)에 업로드합니다.

추론을 실행하려면 파이프라인을 초기화하는 동안 `trust_remote_code` 인수를 추가하여 모든 "마법"을 백그라운드에서 처리합니다.

> [!WARNING]
> `trust_remote_code=True`를 사용하는 추가 예방 조치로, [from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pretrained)의 `revision` 매개변수에 커밋 해시를 전달하여 코드가 악의적인 새 코드 줄로 업데이트되지 않았는지 확인하는 것이 좋습니다(모델 소유자를 완전히 신뢰하지 않는 한).

In [ ]:
from diffusers import DiffusionPipeline
import torch

pipeline = DiffusionPipeline.from_pretrained(
    "<change-username>/<change-id>", trust_remote_code=True, torch_dtype=torch.float16
).to("cuda")

prompt = "hello"

# Text embeds
prompt_embeds, negative_embeds = pipeline.encode_prompt(prompt)

# Keyframes generation (8x64x40, 2fps)
video_frames = pipeline(
    prompt_embeds=prompt_embeds,
    negative_prompt_embeds=negative_embeds,
    num_frames=8,
    height=40,
    width=64,
    num_inference_steps=2,
    guidance_scale=9.0,
    output_type="pt"
).frames

추가 참조로, `trust_remote_code` 기능을 사용하는 [stabilityai/japanese-stable-diffusion-xl](https://huggingface.co/stabilityai/japanese-stable-diffusion-xl/)의 리포지토리 구조를 살펴보세요.

In [ ]:
from diffusers import DiffusionPipeline
import torch

pipeline = DiffusionPipeline.from_pretrained(
    "stabilityai/japanese-stable-diffusion-xl", trust_remote_code=True
)
pipeline.to("cuda")